In [1]:
from signet.trainer.tf_utils.schedules import OneCycleLR
from signet.configs.Conv1D_LSTM_CTC_Loss_tuned_large import Conv1D_LSTM_CTC_Loss
import os
from tqdm import tqdm
import glob
import numpy as np
import pandas as pd
import json
import tensorflow as tf

2023-08-21 10:08:27.043387: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [18]:
label_mask = tf.sequence_mask(2, 10, dtype=tf.int64)
feature_mask = tf.sequence_mask(5, 10, dtype=tf.int64)
tf.stack([label_mask, feature_mask], axis=-1).shape

TensorShape([10, 2])

In [11]:
tensor1 = tf.expand_dims(tf.shape(x)[0], 0)
tensor2 = tf.expand_dims(tf.shape(x)[0], 0)

tensor1

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Reshape
from tensorflow.keras import Model


In [102]:
class PointNetLayer(tf.keras.Model):
    def __init__(self, points_num, feature_dim):
        super(PointNetLayer, self).__init__()

        self.points_num = points_num

        self.conv1 = tf.keras.layers.Conv1D(64, 1, activation='relu')
        self.conv2 = tf.keras.layers.Conv1D(128, 1, activation='relu')
        self.conv3 = tf.keras.layers.Conv1D(1024, 1, activation='relu')
        # self.maxpool = tf.keras.layers.MaxPool1D(points_num)
        self.dense1 = tf.keras.layers.Dense(feature_dim, activation='relu')
        # self.dense2 = tf.keras.layers.Dense(256, activation='relu')
        # self.dense3 = tf.keras.layers.Dense(feature_dim, activation='relu')
        
    def call(self, x):
        batch_size = tf.shape(x)[0]
        frames = tf.shape(x)[1]

        # Reshape the input tensor to have 3 coordinates for each point
        # x = tf.reshape(x, [batch_size, frames * self.points_num, 3])

        # Apply convolutions and max pooling
        net = self.conv1(x)
        net = self.conv2(net)
        net = self.conv3(net)
        net = tf.reshape(net, [batch_size, frames,self.points_num,1024])
        net = tf.reduce_max(net, axis=2)

        # Reshape to separate frames dimension

        # Apply dense layers
        net = self.dense1(net)
        # net = self.dense2(net)
        # net = self.dense3(net)

        return net

In [103]:
layer = PointNetLayer(100,512)

In [105]:
layer(tf.random.uniform((1,10,100,6))).shape

TensorShape([1, 10, 512])

In [96]:
layer(input_tensor,training=True)

<tf.Tensor: shape=(1, 2, 6), dtype=float32, numpy=
array([[[ 0.,  1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10., 11.]]], dtype=float32)>

In [3]:
train_df = pd.read_csv("../dataset/train.csv")
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")
combined = pd.concat([train_df,supp_df])

In [4]:
seqid_to_pid = {}
for seqid,pid in zip(combined.sequence_id,combined.participant_id):
    seqid_to_pid[f"{seqid}.npy"] = pid 

In [5]:
data_root="../dataset/tdf_data"
train_df = pd.read_csv("../dataset/folds/fold3_train.csv")
valid_df = pd.read_csv("../dataset/folds/fold3_valid.csv")

# train_df["pid"] = train_df.files.apply(lambda x: seqid_to_pid[x])
# valid_df["pid"] = valid_df.files.apply(lambda x: seqid_to_pid[x])

# train_labels = set(train_df['pid'].unique())
# valid_labels = set(valid_df['pid'].unique())
# common_labels = train_labels.intersection(valid_labels)

# common_mask = valid_df['pid'].isin(common_labels)

# # train_labels = set(train_df['labels'].unique())
# # valid_labels = set(valid_df['labels'].unique())
# # common_labels = train_labels.intersection(valid_labels)

# # common_mask2 = valid_df['labels'].isin(common_labels)

# # valid_df = valid_df[~(common_mask|common_mask2)]
# valid_df = valid_df[~(common_mask)]

# train_files = train_df.files.apply(lambda x: os.path.join(data_root,x.replace(".npy",".tfrecords")))
valid_files = valid_df.files.apply(lambda x: os.path.join(data_root,x.replace(".npy",".tfrecords")))

In [6]:
len(valid_df)

13004

In [7]:
len(valid_files)

13004

In [8]:
import sys
# sys.path.append("/app/runs/ctc_with_frozenframes/fold3")

In [9]:
from signet.configs.CTC_LOSS_kaggle_copy import ctc_loss_encdec_params
from signet.configs.Conv1D_LSTM_CTC_Loss_tuned_large import Conv1D_LSTM_CTC_Loss

from signet.configs.Encoder_Decoder_Base import EncoderDecoderBase
from signet.dataset.utils import get_ctc_dataset,get_centropy_dataset
from signet.models.feature_extractor import Cnn1dMhsaFeatureExtractor
from signet.models.encoder_decoder import EncoderDecoder

In [10]:
CFG = ctc_loss_encdec_params()
# CFG = EncoderDecoderBase()

In [11]:
CFG.batch_size=128
CFG.max_len=384

In [12]:
dset = get_ctc_dataset(valid_files,CFG,shuffle=False,repeat=False,augment=False)
# dset = get_centropy_dataset(valid_files,CFG,shuffle=False,repeat=False,augment=False)

2023-07-23 10:07:34.187459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21461 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:5e:00.0, compute capability: 7.5
2023-07-23 10:07:35.504197: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_0' with dtype float
	 [[{{node args_0}}]]


In [13]:
model = Cnn1dMhsaFeatureExtractor(CFG)
model.load_weights("/app/runs/ctc_kaggle_copy/fold3/best_ckpt.h5")

# model = EncoderDecoder(CFG)
# model.load_weights("/app/runs/encoder_decoder_base/large_allfold/best_ckpt.h5")

In [14]:
from signet.losses.metrics import normalized_levenshtein_distance,word_accuracy
from signet.trainer.utils import ctc_decode

In [15]:
y_preds = model.predict(dset,steps = len(valid_files)//CFG.batch_size)
y_trues = [label for _, label in dset.unbatch().as_numpy_iterator()]

predictions = []
targets = []
for true_seq,y_pred in zip(y_trues,y_preds):
    pred_seq = ctc_decode(y_pred,CFG.blank_index,CFG.merge_repeated).numpy()
    true_seq = "".join([CFG.idx_to_char[i] for i in true_seq[true_seq!=-1]])
    pred_seq = "".join([CFG.idx_to_char[i] for i in pred_seq[pred_seq!=-1]])
    predictions.append(pred_seq)
    targets.append(true_seq)
nld =  normalized_levenshtein_distance(targets,predictions)

2023-07-23 10:07:46.016325: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [13004]
	 [[{{node Placeholder/_0}}]]
2023-07-23 10:07:46.016951: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [13004]
	 [[{{node Placeholder/_0}}]]
2023-07-23 10:07:48.466399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901


101/101 [==============================] - 27s 241ms/step


2023-07-23 10:08:14.998638: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [13004]
	 [[{{node Placeholder/_0}}]]
2023-07-23 10:08:14.999009: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [13004]
	 [[{{node Placeholder/_0}}]]


In [13]:
import Levenshtein as lev
import plotly.express as px

In [14]:
targets = np.array(targets)
predictions = np.array(predictions)

In [24]:
scores = np.array([lev.distance(p,t)/len(t) for t, p in zip(targets, predictions)])

In [25]:
np.mean(scores)

0.4165386533981755

In [17]:
idx=10
normalized_levenshtein_distance(targets[scores>idx],predictions[scores>idx]),(scores>idx).sum(),(targets[scores>idx]==predictions[scores>idx]).mean()

(0.33224484702260865, 4160, 0.0)

In [36]:
df = pd.read_csv("../dataset/folds/fold3_valid_difficult10.csv")
len(np.intersect1d(df.files,valid_df.files[scores>idx]))/len(df),(scores>idx).sum(),len(df)

(0.875, 1183, 976)

# Error analysis

In [7]:
import pandas as pd
import Levenshtein as lev

In [18]:
PREDICTION_FILE = "/app/runs/ctc_with_frozenframes/fold3/weights/epoch-49-examples.txt"
info = []
with open(PREDICTION_FILE) as f:
    for line in f.readlines():
        info.append(line.strip().split("\t\t"))
df = pd.DataFrame(info,columns=["true","pred"]).dropna()

In [23]:
# Function to apply on every row
def calculate_changes(row):
    true_str = row['true']
    pred_str = row['pred']
    changes = lev.editops( pred_str,true_str)
    return changes

# Creating the new column
df['changes'] = df.apply(calculate_changes, axis=1)

In [33]:
train_df = pd.read_csv("../dataset/train.csv")
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")

In [64]:
allc = {}
for k in train_df["phrase"]:
    if "/" in k:
        break
    for g in k:
        if g not in allc:
            allc[g]=0
        allc[g]+=1

In [68]:

# Extract all replaced characters
insert_chars = {c:0 for c in CFG.char_to_idx}
insert_chars['\\']=0
insert_chars['/']=0
for t,p,change_list in zip(df["true"],df["pred"],df['changes']):
    for change in change_list:
        if change[0] == 'insert':
            insert_chars[t[change[2]]]+=1

# Calculate character frequencies


In [69]:
insert_chars

{' ': 288,
 '!': 0,
 '#': 0,
 '$': 0,
 '%': 0,
 '&': 0,
 "'": 0,
 '(': 0,
 ')': 0,
 '*': 0,
 '+': 23,
 ',': 0,
 '-': 131,
 '.': 97,
 '\\/': 0,
 '0': 88,
 '1': 114,
 '2': 104,
 '3': 112,
 '4': 99,
 '5': 104,
 '6': 88,
 '7': 93,
 '8': 94,
 '9': 88,
 ':': 9,
 ';': 0,
 '=': 5,
 '?': 4,
 '@': 1,
 '[': 0,
 '_': 6,
 'a': 287,
 'b': 40,
 'c': 127,
 'd': 114,
 'e': 244,
 'f': 27,
 'g': 59,
 'h': 75,
 'i': 142,
 'j': 17,
 'k': 55,
 'l': 188,
 'm': 88,
 'n': 208,
 'o': 186,
 'p': 90,
 'q': 9,
 'r': 205,
 's': 132,
 't': 229,
 'u': 111,
 'v': 30,
 'w': 83,
 'x': 6,
 'y': 40,
 'z': 17,
 '~': 0,
 '\\': 90,
 '/': 78}